In [1]:
import pandas as pd
import baseFunctions

In [3]:
dfRaw = pd.read_excel('./ElementarySchools.xlsx')
print(dfRaw.dtypes)
print(dfRaw.shape)

ID                       float64
Overall Rating            object
State Ranking            float64
School Name               object
County                    object
School/Rating Link        object
City                      object
School Address            object
Student Population       float64
Student Teacher Ratio     object
Math Proficiency         float64
Reading Proficiency      float64
Count                    float64
URLRating                 object
URLHomeSearch             object
SortOrder                  int64
Homes for Sale*            int64
Map Link                  object
isSelected               float64
dtype: object
(29, 19)


# Setup

In [4]:
def getContent():
    dfRaw['ResponseHTML'] = [baseFunctions.getContentHTML(x) for x in dfRaw['URLHomeSearch']]
    dfRaw['ResponseSoup'] = [baseFunctions.getContentSoup(x) for x in dfRaw['ResponseHTML']]
    dfValues = pd.DataFrame()
    for index, row in dfRaw.iterrows():
        dfTmp = baseFunctions.getHomeDF(row)
        dfValues = pd.concat([dfValues, dfTmp])
    return(dfValues)

# Get Home Data

In [5]:
valFile = './HistoricalValues.xlsx'
dfValuesHistory = pd.read_excel(valFile)

In [6]:
print(dfValuesHistory.shape)
dfValues = getContent()
print(dfValues.shape)
dfValuesHistory = dfValuesHistory.append(dfValues)
print(dfValuesHistory.shape)

(149, 9)
ERROR: URL = nan
ERROR: URL = nan
ERROR: URL = nan
ERROR: URL = nan
ERROR: URL = nan
ERROR: URL = nan
ERROR: URL = nan
ERROR: URL = nan
ERROR: URL = nan
ERROR: URL = nan
ERROR: URL = nan
ERROR: URL = nan
ERROR: URL = nan
ERROR: URL = nan
(72, 9)
(221, 9)


In [9]:
dfValuesHistory.to_excel(valFile)

# Plots

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [7]:
#valW = 16
#valH = 12
#fig, ax = plt.subplots(figsize=(valW,valH))
#bp = dfValuesHistory.groupby('AddrStreet').plot(x='DateRetrieved',y='Price',ax=ax)
#ax.legend_ = None
#plt.title("Plot of House Prices Over Time")

In [8]:
dfSummary = dfValuesHistory.drop_duplicates(['AddrStreet','ElementaryID'],keep='first')
dfCnts = dfSummary.groupby(['ElementaryID']).count()['AddrStreet'].to_frame().reset_index().sort_values(['AddrStreet'],ascending=False)
dfCnts.columns = ['ElementaryID','CountHomes']
dfDRMin = dfSummary.groupby(['ElementaryID']).min()['DateRetrieved'].to_frame().reset_index()
dfDRMin.columns = ['ElementaryID','DateMin']
dfDRMax = dfSummary.groupby(['ElementaryID']).max()['DateRetrieved'].to_frame().reset_index()
dfDRMax.columns = ['ElementaryID','DateMax']
dfPriceMin = dfSummary.groupby(['ElementaryID']).min()['Price'].to_frame().reset_index()
dfPriceMin.columns = ['ElementaryID','PriceMin']
dfPriceMax = dfSummary.groupby(['ElementaryID']).max()['Price'].to_frame().reset_index()
dfPriceMax.columns = ['ElementaryID','PriceMax']
dfCnts = pd.merge(left=dfCnts,right=dfDRMin,on='ElementaryID',how='left')
dfCnts = pd.merge(left=dfCnts,right=dfDRMax,on='ElementaryID',how='left')
dfCnts = pd.merge(left=dfCnts,right=dfPriceMin,on='ElementaryID',how='left')
dfCnts = pd.merge(left=dfCnts,right=dfPriceMax,on='ElementaryID',how='left')
print(dfCnts.dtypes)

ElementaryID           float64
CountHomes               int64
DateMin         datetime64[ns]
DateMax         datetime64[ns]
PriceMin                 int64
PriceMax                 int64
dtype: object


In [9]:
dfFinal = pd.merge(left=dfRaw,right=dfCnts,left_on='ID',right_on='ElementaryID',how='left')
print(dfFinal.shape)

(29, 25)


In [10]:
dfFinal.to_excel('./FinalCnts.xlsx')